In [ ]:
# -*- coding: utf-8 -*-
"""Imbalanced_Classes.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1GTN2Q9XyFLIyex-nmuU7PB8k4tUmSW-q
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import re
import csv


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import os
# print(os.listdir("../input"))
plt.style.use('ggplot')

df = pd.DataFrame()

raw = open(r"C:\\Users\\z3696\\Documents\\Document-Classification\\classifier\\NIST_FULL\\2019-neg.txt", encoding="ISO-8859-1")
lines = raw.readlines()
raw.close()

# remove /n at the end of each line
for index, line in enumerate(lines):
    lines[index] = line.strip()
    print(lines[index])

neg_2019_df = pd.DataFrame(columns=['sentence'])
i = 0
first_col = ""
for line in lines:
    first_col = re.sub(r' \(.*', "", line)
    neg_2019_df.loc[i] = [first_col]
    i = i+1

neg_2019_df.head()
neg_2019_df['label'] = 0
#print(neg_2019_df)

raw1 = open(r"C:\\Users\\z3696\\Documents\\Document-Classification\\classifier\\NIST_FULL\\2019-pos.txt", encoding="ISO-8859-1")
lines1 = raw1.readlines()
raw1.close()

# remove /n at the end of each line
for index, line in enumerate(lines1):
    lines1[index] = line.strip()
    print(lines1[index])

pos_2019_df = pd.DataFrame(columns=['sentence'])
i = 0
first_col = ""
for line in lines1:
    first_col = re.sub(r' \(.*', "", line)
    pos_2019_df.loc[i] = [first_col]
    i = i+1

pos_2019_df.head()
pos_2019_df['label'] = 1
#print(pos_2019_df)

df = df.append(pos_2019_df)
df = df.append(neg_2019_df)
print(df)
df.shape

import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support as score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
stopwords.words('english')

#Removing punctuations from entire dataset
punc_set = string.punctuation
punc_set

#Function for removing punctions
def remove_punc(text):
    clean = "".join([x.lower() for x in text if x not in punc_set])
    return clean

#Applying the 'remove_punc' function to entire dataset
df['no_punc'] = df['sentence'].apply(lambda z:remove_punc(z))

#Function for Tokenizing entire data for representing every word as datapoint
def tokenize(text):
    tokens = re.split("\W+",text)
    return tokens

#Applying the 'tokenize' function to entire dataset
df['tokenized_Data'] = df['no_punc'].apply(lambda z:tokenize(z))

#Importing stopwords from NLTK Library to remove stopwords now that we have tokenized it
stopwords = nltk.corpus.stopwords.words('english')

#Function for removing stopwords from single row
def remove_stopwords(tokenized_words):
    Ligit_text=[word for word in tokenized_words if word not in stopwords]
    return Ligit_text

#Applying the function 'remove_stopwords' from the entire dataset
df["no_stop"] = df["tokenized_Data"].apply(lambda z:remove_stopwords(z))

#Importing 'WordNetLemmatizer' as lemmatizing function to find lemma's of words
wnl = nltk.wordnet.WordNetLemmatizer()

#Function for lemmatizing the tokenzied text
def lemmatizing(tokenized_text):
    lemma = [wnl.lemmatize(word) for word in tokenized_text]
    return lemma

#Applying the 'lemmatizing' function to entire dataset
df['lemmatized'] = df['no_stop'].apply(lambda z:lemmatizing(z))

# #Importing the 'SnowballStemmer' and declaring variable 'sno' to save the stemmer in.
# #This Stemmer gives slightly better results as compared to 'PorterStemmer'
# sno = nltk.SnowballStemmer('english')

# #Function for applying stemming to find stem roots of all words
# def stemming(tokenized_text):
#     text= [sno.stem(word) for word in tokenized_text]
#     return text

# #Applying the 'stemming' function to entire dataset
# data['ss_stemmed'] = data['lemmatized'].apply(lambda z:stemming(z))


# ps = nltk.PorterStemmer()

# def stemming(tokenized_text):
#     text= [ps.stem(word) for word in tokenized_text]
#     return text

# data['ps_stemmed'] = data['lemmatized'].apply(lambda z:stemming(z))

#This step is done here because, the 'lemmatized' column is a list of tokenized words and when we apply vectorization
#techniques such as count vectorizer or TFIDF, they require string input. Hence convert all tokenzied words to string
df['lemmatized'] = [" ".join(review) for review in df['lemmatized'].values]

df.head()

#Splitting data into smaller dataframes for the purpose of Training and Testing
x1 = df.iloc[1:47990,5]
x2 = df.iloc[47991:95980,5]
y1 = df.iloc[1:47990,1]
y2 = df.iloc[47991:95980,1]
#x_seg = df.iloc[0:5,5]
#y_seg = df.iloc[0:5,1]
print(x1.shape)
print(x2.shape)
print(y1.shape)
print(y2.shape)

'''
x = df['lemmatized'].values
y = df['label'].values
print(x.shape)
print(y.shape)
'''

#Declaring and applying TFIDF functions to train and test data
tfidf_vect = TfidfVectorizer(ngram_range=(1,2))
tfidf_train = tfidf_vect.fit_transform(x1.values)
tfidf_test=tfidf_vect.transform(x2.values)
print(tfidf_train.shape)
print(tfidf_test.shape)
#tfidf_train.toarray()

#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

tfidf_vect = TfidfVectorizer()
x_tfidf = tfidf_vect.fit_transform(df["lemmatized"])

x_train, x_test, y_train, y_test = train_test_split(x_tfidf,df["label"],test_size=0.5)

log = LogisticRegression(penalty='l2',random_state=0, solver='lbfgs', multi_class='auto', max_iter=500)
model_lr = log.fit(x_train,y_train)
probs_lr = model_lr.predict_proba(x_test)[:, 1]
ly_prediction = log.predict(x_test)
fly = f1_score(ly_prediction,y_test)
print("===Logistic Regression with TfidfVectorizer Imbalanced - 2019===")
print('Logistic F1-score',fly*100)
print('Logistic ROCAUC score:',roc_auc_score(y_test, ly_prediction)*100)
print('Logistic Recall score:', recall_score(y_test, ly_prediction)*100)
print('Logistic Precision Score:', precision_score(y_test, ly_prediction)*100)
print('Logistic Confusion Matrix', confusion_matrix(y_test,ly_prediction), "\n")
print('Logistic Classification', classification_report(y_test,ly_prediction), "\n")
print('Logistic Accuracy Score', accuracy_score(y_test, ly_prediction)*100)

from sklearn.tree import DecisionTreeClassifier
DCT = DecisionTreeClassifier()
model_dt = DCT.fit(x_train, y_train)
probs_dt = model_dt.predict_proba(x_test)[:, 1]
dct_pred = DCT.predict(x_test)
fdct = f1_score(dct_pred,y_test)
print("===DecisionTreeClassifier with TfidfVectorizer Imbalanced - 2019===")
print('DCT F1-score',fdct*100)
print('DCT ROCAUC score:',roc_auc_score(y_test, dct_pred)*100)
print('DCT Recall score:', recall_score(y_test, dct_pred)*100)
print('DCT Precision Score:', precision_score(y_test, dct_pred)*100)
print('DCT Confusion Matrix', confusion_matrix(y_test, dct_pred), "\n")
print('DCT Classification', classification_report(y_test, dct_pred), "\n")
print('DCT Accuracy Score', accuracy_score(y_test, dct_pred)*100)

from sklearn.naive_bayes import MultinomialNB
Naive = MultinomialNB()
model_nb = Naive.fit(x_train,y_train)
probs_nb = model_nb.predict_proba(x_test)[:, 1]
# predict the labels on validation dataset
ny_pred = Naive.predict(x_test)
fna = f1_score(ny_pred,y_test)
# Use accuracy_score function to get the accuracy
print("===Naive Bayes with TfidfVectorizer Imabalanced - 2019 ===")
print('Naive F1-score',fna*100)
print('Naive ROCAUC score:',roc_auc_score(y_test, ny_pred)*100)
print('Naive Recall score:', recall_score(y_test, ny_pred)*100)
print('Naive Precision Score:', precision_score(y_test, ny_pred)*100)
print('Naive Confusion Matrix', confusion_matrix(y_test, ny_pred), "\n")
print('Naive Classification', classification_report(y_test, ny_pred), "\n")
print('Naive Accuracy Score', accuracy_score(y_test, ny_pred)*100)

# XGBoost Classifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
xgb_model = XGBClassifier().fit(x_train, y_train)
probs_xg = xgb_model.predict_proba(x_test)[:, 1]
# predict
xgb_y_predict = xgb_model.predict(x_test)
fxg = f1_score(xgb_y_predict,y_test)
print("===XGB with TfidfVectorizer Imbalanced - 2019===")
print('XGB F1-Score', fxg*100)
print('XGB ROCAUC Score:', roc_auc_score(xgb_y_predict, y_test)*100)
print('XGB Recall score:', recall_score(xgb_y_predict, y_test)*100)
print('XGB Precision Score:', precision_score(xgb_y_predict, y_test)*100)
print('XGB Confusion Matrix', confusion_matrix(xgb_y_predict, y_test), "\n")
print('XGB Classification', classification_report(xgb_y_predict, y_test), "\n")
print('XGB Accuracy Score', accuracy_score(xgb_y_predict, y_test)*100)
'''
# Support Vector Machine Classifier
from sklearn.svm import SVC
csvm_model = SVC(C=1.0, kernel='linear', degree=3, gamma='auto').fit(x_train,y_train)
probs_cs = csvm_model.predict_proba(x_test)[:, 1]
# predict the labels on validation dataset
csvy_pred = csvm_model.predict(x_test)
fcsvm = f1_score(csvy_pred,y_test)
print("===C-SVM with TfidfVectorizer Imbalanced - 2019===")
print('C-SVM F1-score',fcsvm*100)
# Use accuracy_score function to get the accuracy
print('C-SVM ROCAUC score:',roc_auc_score(y_test, csvy_pred)*100, "\n")
print('C-SVM Recall score:', recall_score(y_test, csvy_pred)*100, "\n")
print('C-SVM Precision Score:', precision_score(y_test, csvy_pred)*100, "\n")
print('C-SVM Confusion Matrix', confusion_matrix(y_test, csvy_pred), "\n")
print('C-SVM Classification', classification_report(y_test, csvy_pred), "\n")
print('C-SVM Accuracy Score', accuracy_score(y_test, csvy_pred)*100, "\n")
'''
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(n_estimators=1000, random_state=0).fit(x_train,y_train)
probs_rf = rfc_model.predict_proba(x_test)[:, 1]
rfc_pred = rfc_model.predict(x_test)
frfc = f1_score(rfc_pred,y_test)
print("====RandomForest with Tfidf Imbalanced 2019====")
print('RFC F1 score', frfc*100)
print('RFC ROCAUC Score:', roc_auc_score(y_test, rfc_pred)*100)
print('RFC Recall score:', recall_score(y_test, rfc_pred)*100)
print('RFC Precision Score:', precision_score(y_test, rfc_pred)*100)
print('RFC Confusion Matrix', confusion_matrix(y_test,rfc_pred), "\n")
print('RFC Classification', classification_report(y_test,rfc_pred), "\n")
print('RFC Accuracy Score', accuracy_score(y_test,rfc_pred)*100)


from sklearn.metrics import auc, precision_recall_curve

y_test_int = y_test.replace({'Positive': 1, 'Negative': 0})

baseline_model = sum(y_test_int == 1) / len(y_test_int)

precision_lr, recall_lr, _ = precision_recall_curve(y_test_int, probs_lr)
auc_lr = auc(recall_lr, precision_lr)

precision_dt, recall_dt, _ = precision_recall_curve(y_test_int, probs_dt)
auc_dt = auc(recall_dt, precision_dt)

precision_nb, recall_nb, _ = precision_recall_curve(y_test_int, probs_nb)
auc_nb = auc(recall_nb, precision_nb)

precision_xg, recall_xg, _ = precision_recall_curve(y_test_int, probs_xg)
auc_xg = auc(recall_xg, precision_xg)
'''
precision_cs, recall_cs, _ = precision_recall_curve(y_test_int, probs_cs)
auc_cs = auc(recall_cs, precision_cs)
'''
precision_rf, recall_rf, _ = precision_recall_curve(y_test_int, probs_rf)
auc_rf  = auc(recall_rf, precision_rf)
'''
plt.figure(figsize=(12, 7))
plt.plot([0, 1], [baseline_model, baseline_model], linestyle='--', label='Baseline model')
plt.plot(recall_lr, precision_lr, label=f'AUC Log. Reg. Imb.) = {auc_lr:.2f}')
plt.plot(recall_dt, precision_dt, label=f'AUC Dec. Tree Imb.) = {auc_dt:.2f}')
plt.plot(recall_nb, precision_nb, label=f'AUC Nai Bay. Imb.) = {auc_rf:.2f}')
plt.plot(recall_xg, precision_xg, label=f'AUC XGB Imb.) = {auc_xg:.2f}')
'''
'''
plt.plot(recall_lr1, precision_lr1, label=f'AUC Log. Reg. ROS) = {auc_lr:.2f}')
plt.plot(recall_dt1, precision_dt1, label=f'AUC Dec. Tree ROS) = {auc_dt:.2f}')
plt.plot(recall_nb1, precision_nb1, label=f'AUC Nai Bay. ROS) = {auc_rf:.2f}')
plt.plot(recall_xg1, precision_xg1, label=f'AUC XGB ROS) = {auc_xg:.2f}')
plt.plot(recall_lr2, precision_lr2, label=f'AUC Log. Reg. ROS) = {auc_lr2:.2f}')
plt.plot(recall_dt2, precision_dt2, label=f'AUC Dec. Tree ROS) = {auc_dt2:.2f}')
plt.plot(recall_nb2, precision_nb2, label=f'AUC Nai Bay. ROS) = {auc_rf2:.2f}')
plt.plot(recall_xg2, precision_xg2, label=f'AUC XGB ROS) = {auc_xg2:.2f}')

plt.title('Precision-Recall Curves 2019: Imbalanced', size=20)
plt.xlabel('Recall', size=14)
plt.ylabel('Precision', size=14)
plt.legend()
plt.show();
'''
df['label'].value_counts()

import seaborn as sns
g = sns.countplot(df['label'])
g.set_xticklabels(['Negative','Positive'])
plt.show()

# class count
label_count_neg, label_count_pos = df['label'].value_counts()

# Separate class
label_neg = df[df['label'] == 0]
label_pos = df[df['label'] == 1]# print the shape of the class
print('Label Negative:', label_neg.shape)
print('Label Positive:', label_pos.shape)

label_neg_under = label_neg.sample(label_count_pos)

test_under = pd.concat([label_neg_under, label_pos], axis=0)

print("total class of pos and neg :",test_under['label'].value_counts())# plot the count after under-sampeling
test_under['label'].value_counts().plot(kind='bar', title='label (target)')

label_pos_over = label_pos.sample(label_count_neg, replace=True)

test_over = pd.concat([label_pos_over, label_neg], axis=0)

print("total class of pos and neg:",test_under['label'].value_counts())# plot the count after under-sampeling
test_over['label'].value_counts().plot(kind='bar', title='label (target)')

import imblearn

# import library
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
x = x_tfidf
y = df['label']
print(x.shape)
print(y.shape)


rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(x, y)

print('Original dataset shape:', Counter(y))
print('Resample dataset shape', Counter(y_rus))

# import library
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
x =x_tfidf
y = df["label"]
rus = RandomUnderSampler(random_state=42)
#RUS = RandomOverSampler(random_state=42)
#Random over-sampling with imblearn
# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_rus))
#Random over-sampling with imblearn
ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
x_ros, y_ros = ros.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))
# import library

# Under-sampling: Tomek links
from imblearn.under_sampling import TomekLinks
from collections import Counter


tl = RandomOverSampler(sampling_strategy='majority')

# fit predictor and target variable
x_tl, y_tl = ros.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

# import library
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))


from imblearn.under_sampling import NearMiss

nm = NearMiss()

x_nm, y_nm = nm.fit_resample(x, y)

print('Original dataset shape:', Counter(y))
print('Resample dataset shape:', Counter(y_nm))

#====RandomUnderSampler=====
x1_train, x1_test, y1_train, y1_test = train_test_split(x_tl,y_tl,test_size=0.5)

log = LogisticRegression(penalty='l2',random_state=0, solver='lbfgs', multi_class='auto', max_iter=500)
model_lr1 = log.fit(x1_train,y1_train)
probs_lr1 = model_lr1.predict_proba(x1_test)[:, 1]
ly_prediction = log.predict(x1_test)
fly = f1_score(ly_prediction,y1_test)
print("===Logistic Regression with TfidfVectorizer Tomelinks - 2019===")
print('Logistic F1-score',fly*100)
print('Logistic ROCAUC score:',roc_auc_score(y1_test, ly_prediction)*100)
print('Logistic Recall score:', recall_score(y1_test, ly_prediction)*100)
print('Logistic Precision Score:', precision_score(y1_test, ly_prediction)*100)
print('Logistic Confusion Matrix', confusion_matrix(y1_test,ly_prediction), "\n")
print('Logistic Classification', classification_report(y1_test,ly_prediction), "\n")
print('Logistic Accuracy Score', accuracy_score(y1_test, ly_prediction)*100)

from sklearn.tree import DecisionTreeClassifier
DCT = DecisionTreeClassifier()
model_dt1 = DCT.fit(x1_train, y1_train)
probs_dt1 = model_dt1.predict_proba(x1_test)[:, 1]
dct_pred = DCT.predict(x1_test)
fdct = f1_score(dct_pred,y1_test)
print("===DecisionTreeClassifier with TfidfVectorizer Tomelinks - 2019===")
print('DCT F1-score',fdct*100)
print('DCT ROCAUC score:',roc_auc_score(y1_test, dct_pred)*100)
print('DCT Recall score:', recall_score(y1_test, dct_pred)*100)
print('DCT Precision Score:', precision_score(y1_test, dct_pred)*100)
print('DCT Confusion Matrix', confusion_matrix(y1_test, dct_pred), "\n")
print('DCT Classification', classification_report(y1_test, dct_pred), "\n")
print('DCT Accuracy Score', accuracy_score(y1_test, dct_pred)*100)

from sklearn.naive_bayes import MultinomialNB
Naive = MultinomialNB()
model_nb1 = Naive.fit(x1_train,y1_train)
probs_nb1 = model_nb1.predict_proba(x1_test)[:, 1]
# predict the labels on validation dataset
ny_pred = Naive.predict(x1_test)
fna = f1_score(ny_pred,y1_test)
# Use accuracy_score function to get the accuracy
print("===Naive Bayes with TfidfVectorizer Tomelinks - 2019 ===")
print('Naive F1-score',fna*100)
print('Naive ROCAUC score:',roc_auc_score(y1_test, ny_pred)*100)
print('Naive Recall score:', recall_score(y1_test, ny_pred)*100)
print('Naive Precision Score:', precision_score(y1_test, ny_pred)*100)
print('Naive Confusion Matrix', confusion_matrix(y1_test, ny_pred), "\n")
print('Naive Classification', classification_report(y1_test, ny_pred), "\n")
print('Naive Accuracy Score', accuracy_score(y1_test, ny_pred)*100)

# XGBoost Classifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
XGB = XGBClassifier()
xgb_model1 = XGB.fit(x1_train,y1_train)
probs_xg1 = xgb_model1.predict_proba(x1_test)[:, 1]
# predict
xgb_y_predict = xgb_model1.predict(x1_test)
fxg = f1_score(xgb_y_predict,y1_test)
print("===XGB with TfidfVectorizer Tomelinks- 2019===")
print('XGB F1-Score', fxg*100)
print('XGB ROCAUC Score:', roc_auc_score(xgb_y_predict, y1_test)*100)
print('XGB Recall score:', recall_score(xgb_y_predict, y1_test)*100)
print('XGB Precision Score:', precision_score(xgb_y_predict, y1_test)*100)
print('XGB Confusion Matrix', confusion_matrix(xgb_y_predict, y1_test), "\n")
print('XGB Classification', classification_report(xgb_y_predict, y1_test), "\n")
print('XGB Accuracy Score', accuracy_score(xgb_y_predict, y1_test)*100)
'''
# Support Vector Machine Classifier
from sklearn.svm import SVC
csvm_model1 = SVC(C=1.0, kernel='linear', degree=3, gamma='auto').fit(x_train,y_train)
probs_cs1 = csvm_model1.predict_proba(x1_test)[:, 1]
# predict the labels on validation dataset
csvy_pred = csvm_model1.predict(x1_test)
fcsvm = f1_score(csvy_pred,y1_test)
print("===C-SVM with TfidfVectorizer Tomelinks - 2019===")
print('C-SVM F1-score',fcsvm*100)
# Use accuracy_score function to get the accuracy
print('C-SVM ROCAUC score:',roc_auc_score(y1_test, csvy_pred)*100, "\n")
print('C-SVM Recall score:', recall_score(y1_test, csvy_pred)*100, "\n")
print('C-SVM Precision Score:', precision_score(y1_test, csvy_pred)*100, "\n")
print('C-SVM Confusion Matrix', confusion_matrix(y1_test, csvy_pred), "\n")
print('C-SVM Classification', classification_report(y1_test, csvy_pred), "\n")
print('C-SVM Accuracy Score', accuracy_score(y1_test, csvy_pred)*100, "\n")
'''
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc_model1 = RandomForestClassifier(n_estimators=1000, random_state=0).fit(x_train,y_train)
probs_rf1 = rfc_model.predict_proba(x1_test)[:, 1]
rfc_pred = rfc_model.predict(x1_test)
frfc = f1_score(rfc_pred,y1_test)
print("====RandomForest with Tfidf Tomelinks-  2019====")
print('RFC F1 score', frfc*100)
print('RFC ROCAUC Score:', roc_auc_score(y1_test, rfc_pred)*100)
print('RFC Recall score:', recall_score(y1_test, rfc_pred)*100)
print('RFC Precision Score:', precision_score(y1_test, rfc_pred)*100)
print('RFC Confusion Matrix', confusion_matrix(y1,rfc_pred), "\n")
print('RFC Classification', classification_report(y1_test,rfc_pred), "\n")
print('RFC Accuracy Score', accuracy_score(y1_test,rfc_pred)*100)

y_test_int1 = y1_test.replace({'Positive': 1, 'Negative': 0})

precision_lr1, recall_lr1, _ = precision_recall_curve(y_test_int1, probs_lr1)
auc_lr1 = auc(recall_lr1, precision_lr1)

precision_dt1, recall_dt1, _ = precision_recall_curve(y_test_int1, probs_dt1)
auc_dt1 = auc(recall_dt1, precision_dt1)

precision_nb1, recall_nb1, _ = precision_recall_curve(y_test_int1, probs_nb1)
auc_nb1 = auc(recall_nb1, precision_nb1)

precision_xg1, recall_xg1, _ = precision_recall_curve(y_test_int1, probs_xg1)
auc_xg1 = auc(recall_xg1, precision_xg1)
'''
precision_cs1, recall_cs1, _ = precision_recall_curve(y_test_int1, probs_cs1)
auc_cs1 = auc(recall_cs1, precision_cs1)
'''
precision_rf1, recall_rf1, _ = precision_recall_curve(y_test_int1, probs_rf1)
auc_rf1 = auc(recall_rf1, precision_rf1)

'''
plt.plot(recall_lr1, precision_lr1, label=f'AUC Log. Reg. ROS) = {auc_lr:.2f}')
plt.plot(recall_dt1, precision_dt1, label=f'AUC Dec. Tree ROS) = {auc_dt:.2f}')
plt.plot(recall_nb1, precision_nb1, label=f'AUC Nai Bay. ROS) = {auc_rf:.2f}')
plt.plot(recall_xg1, precision_xg1, label=f'AUC XGB ROS) = {auc_xg:.2f}')
'''

plt.figure(figsize=(12, 7))
plt.plot([0, 1], [baseline_model, baseline_model], linestyle='--', label='Baseline model')
plt.plot(recall_lr, precision_lr, label=f'AUC Log. Reg. Imb. = {auc_lr:.2f}')
plt.plot(recall_dt, precision_dt, label=f'AUC Dec. Tree Imb. = {auc_dt:.2f}')
plt.plot(recall_nb, precision_nb, label=f'AUC Nai Bay. Imb. = {auc_nb:.2f}')
plt.plot(recall_xg, precision_xg, label=f'AUC XGB Imb. = {auc_xg:.2f}')
#plt.plot(recall_cs, precision_cs, label=f'AUC CSVM Imb. = {auc_cs:.2f}')
plt.plot(recall_rf, precision_rf, label=f'AUC RFC Imb. = {auc_rf:.2f}')
plt.plot(recall_lr1, precision_lr1, label=f'AUC Log. Reg. Tomelinks = {auc_lr1:.2f}')
plt.plot(recall_dt1, precision_dt1, label=f'AUC Dec. Tree Tomelinks = {auc_dt1:.2f}')
plt.plot(recall_nb1, precision_nb1, label=f'AUC Nai Bay. Tomelinks = {auc_nb1:.2f}')
plt.plot(recall_xg1, precision_xg1, label=f'AUC XGB Tomelinks = {auc_xg1:.2f}')
#plt.plot(recall_cs1, precision_cs1, label=f'AUC CSVM Tomelinks = {auc_cs1:.2f}')
plt.plot(recall_rf1, precision_rf1, label=f'AUC RFC Tomelinks = {auc_rf1:.2f}')
plt.title('Precision-Recall Curves 2019: Imbalanced/Tomelinks', size=20)
plt.xlabel('Recall', size=14)
plt.ylabel('Precision', size=14)
plt.legend()
plt.show();

'''
plt.plot(recall_lr2, precision_lr2, label=f'AUC Log. Reg. ROS) = {auc_lr2:.2f}')
plt.plot(recall_dt2, precision_dt2, label=f'AUC Dec. Tree ROS) = {auc_dt2:.2f}')
plt.plot(recall_nb2, precision_nb2, label=f'AUC Nai Bay. ROS) = {auc_rf2:.2f}')
plt.plot(recall_xg2, precision_xg2, label=f'AUC XGB ROS) = {auc_xg2:.2f}')
'''
#====RandomOverSampler====
'''

'''
'''
# import library
from imblearn.under_sampling import TomekLinks
from collections import Counter


tl = RandomOverSampler(sampling_strategy='majority')

# fit predictor and target variable
x_tl, y_tl = ros.fit_resample(x, y)


print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

# import library
from imblearn.under_sampling import TomekLinks
from collections import Counter

tl = RandomOverSampler(sampling_strategy='majority')

# fit predictor and target variable
x_tl, y_tl = ros.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

# import library
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

from imblearn.under_sampling import SMOTE

nm = SMOTE()

x_nm, y_nm = nm.fit_resample(x, y)

print('Original dataset shape:', Counter(y))
print('Resample dataset shape:', Counter(y_nm))

x1_train, x1_test, y1_train, y1_test = train_test_split(x_smote,y_smote,test_size=0.5)

xgb_predict = xgb_model.predict(x1_test)
fxg = f1_score(xgb_predict,y1_test)
print("===XGB Balanced - 2019 SMOTE===")
print('XGB F1-Score', fxg*100)
print('XGB ROCAUC Score:', roc_auc_score(xgb_predict, y1_test), "\n")
print('XGB Confusion Matrix', confusion_matrix(xgb_predict, y1_test), "\n")
print('XGB Classification', classification_report(xgb_predict, y1_test), "\n")
print('XGB Accuracy Score', accuracy_score(xgb_predict, y1_test)*100)

from sklearn.naive_bayes import MultinomialNB
Naive = MultinomialNB()
Naive.fit(x1_train,y1_train)
# predict the labels on validation dataset
nb_pred = Naive.predict(x1_test)
fna = f1_score(nb_pred,y1_test)
# Use accuracy_score function to get the accuracy
print("===Naive Bayes Balanced - 2019 SMOTE===")
print('Naive F1-score',fna*100)
print('Naive ROCAUC score:',roc_auc_score(y1_test, nb_pred)*100, "\n")
print('Naive Confusion Matrix', confusion_matrix(y1_test, nb_pred), "\n")
print('Naive Classification', classification_report(y1_test, nb_pred), "\n")
print('Naive Accuracy Score', accuracy_score(y1_test, nb_pred)*100)

from sklearn.tree import DecisionTreeClassifier
Tree = DecisionTreeClassifier()
Tree.fit(x1_train, y1_train)
tree_pred = Tree.predict(x1_test)
ftree = f1_score(tree_pred,y1_test)
print("===Decision Tree Balanced - 2019 SMOTE ===")
print('DCT F1-score', ftree*100)
print('DCT ROCAUC score:',roc_auc_score(y1_test, tree_pred)*100, "\n")
print('DCT Confusion Matrix', confusion_matrix(y1_test, tree_pred), "\n")
print('DCT Classification', classification_report(y1_test, tree_pred), "\n")
print('DCT Accuracy Score', accuracy_score(y1_test, tree_pred)*100)

lR = LogisticRegression(penalty='l2',random_state=0, solver='lbfgs', multi_class='auto', max_iter=500)
lR.fit(x1_train,y1_train)
lr_y_prediction = lR.predict(x1_test)
fly = f1_score(lr_y_prediction,y1_test)
print("===Logistic Regression - 2019 SMOTE ===")
print('Logistic F1-score',fly*100)
print('Logistic ROCAUC score:',roc_auc_score(y1_test, lr_y_prediction))
print('Logistic Confusion Matrix', confusion_matrix(y1_test,lr_y_prediction), "\n")
print('Logistic Classification', classification_report(y1_test,lr_y_prediction), "\n")
print('Logistic Accuracy Score', accuracy_score(y1_test,lr_y_prediction)*100)

#LinearSVC Classifier
from sklearn.svm import LinearSVC
lsvc_model = LinearSVC()
lsvc_model.fit(x1_train,y1_train)
lsvcm_pred = lsvc_model.predict(x1_test)
lsvccm = f1_score(lsvc_pred,y1_test)
print("=== LSVC - 2019 SMOTE ===")
print('LSVC F1 score', lsvccm*100)
print('LSVC ROCAUC Score:', roc_auc_score(y1_test, lsvcm_pred)*100, "\n")
print('LSVC Confusion Matrix', confusion_matrix(y1_test, lsvcm_pred), "\n")
print('LSVC Classification', classification_report(y1_test, lsvcm_pred), "\n")
print('LSVC Accuracy Score', accuracy_score(y1_test, lsvcm_pred)*100)

'''
'''
# load library
from sklearn.ensemble import RandomForestClassifier

rfc1 = RandomForestClassifier()

# fit the predictor and target
rfc1.fit(x1_train, y1_train)

# predict
rfc_predict = rfc1.predict(x1_test)# check performance
print("=== RFC Balanced - 2019 SMOTE  ===")
print('RFC ROCAUC score:',roc_auc_score(y1_test, rfc_predict)*100)
print('RFC Accuracy score:',accuracy_score(y1_test, rfc_predict)*100)
print('RFC F1 score:',f1_score(y1_test, rfc_predict)*100)
print('RFC Confusion Matrix', confusion_matrix(y1_test, rfc_predict), "\n")
print('RFC Classification', classification_report(y1_test, rfc_predict), "\n")

# load library
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# we can add class_weight='balanced' to add panalize mistake
csvc_model = SVC(class_weight='balanced', probability=True)

csvc_model.fit(x1_train, y1_train)

csvc_predict = csvc_model.predict(x1_test)# check performance
print("====CSVM Balanced - 2019 SMOTE ====")
print('CSVM ROCAUC score:',roc_auc_score(y1_test, csvc_predict)*100)
print('CSVM Accuracy score:',accuracy_score(y1_test, csvc_predict)*100)
print('CSVM F1 score:',f1_score(y1_test, csvc_predict)*100)
print('CSVM Confusion Matrix', confusion_matrix(y1_test, csvc_predict), "\n")
print('CSVM Classification', classification_report(y1_test, csvc_predict), "\n")
'''
'''
#Multiple models
models = [KNeighborsClassifier(),
          LogisticRegression(solver='lbfgs', multi_class='ovr'),
          DecisionTreeClassifier(),
          SVC(gamma='scale'),
          RandomForestClassifier(n_estimators=100),
          ExtraTreesClassifier(n_estimators=100)]

tvec = TfidfVectorizer(stop_words='english',
                       #sublinear_tf=True,
                       max_df=0.5,
                       max_features=1000)

tvec.fit(data_train['data'])
X_train = tvec.transform(data_train['data'])
X_test = tvec.transform(data_test['data'])

res = []

for model in models:
    print(model)
    print()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)
    print()
    cm = docm(y_test, y_pred, data_train.target_names)
    print(cm)
    res.append([model, score])
    print()
    print('-'*60)
    print()

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay)

import matplotlib.pyplot as plt
#predictions = clf.predict(X_test)
precision, recall, _ = precision_recall_curve(y_test, y_pred)

disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.show()
'''
'''
plot_precision_recall_curve(xgb_model, x1_test, y1_test, ax = plt.gca(),name = "XGB - SMOTE")
plot_precision_recall_curve(Naive, x1_test, y1_test, ax = plt.gca(),name = "Naive Bayes - SMOTE")
plot_precision_recall_curve(Tree, x1_test, y1_test, ax = plt.gca(),name = "Decision Tree - SMOTE")
plot_precision_recall_curve(lR, x1_test, y1_test, ax = plt.gca(),name = "Logistic Regression - SMOTE")
plt.title('Precision-Recall curve for SMOTE')
'''